# Menganalisis Risiko Gagal Bayar Peminjam

## 1. File Data dan Informasi umum

In [ ]:
import pandas as pd

In [ ]:
try:
  df = pd.read_csv('/content/credit_scoring_eng.csv')
except:
  df = pd.read_csv('/datasets/credit_scoring_eng.scv')

  #data credit scoring akan di simpan dalam variabel df

### Soal 1. Eksplorasi data

**Deskripsi Data**
- `children` - jumlah anak dalam keluarga
- `days_employed` - pengalaman kerja nasabah dalam hari
- `dob_years` - usia nasabah dalam tahun
- `education` - tingkat pendidikan nasabah
- `education_id` - pengidentifikasi untuk tingkat pendidikan nasabah
- `family_status` - pengidentifikasi untuk status perkawinan nasabah
- `family_status_id` - tanda pengenal status perkawinan
- `gender` - jenis kelamin nasabah
- `income_type` - jenis pekerjaan
- `debt` - apakah nasabah memiliki hutang pembayaran pinjaman
- `total_income` - pendapatan bulanan
- `purpose` - tujuan mendapatkan pinjaman

In [ ]:
df.shape 

(21525, 12)

data memiliki 21525 baris dan 12 kolom

In [ ]:
df.head(1)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house


Di kolom **days_employed** memiliki nilai minus(-) dan bertype float, yang seharusnya bertipe integer dikarenakan days_employed adalah pengalaman kerja dalam hari 

In [ ]:
df.dtypes

children              int64
days_employed       float64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Jumlah total baris adalah 21525, tetapi  di bagian kolom **days_employed dan total income** memiliki nilai yang hilang.

sekarang diperlukan filtering atau melihat lebih lanjut data yang hilang tersebut disengaja atau tidak. dikarenakan pada kolom days_employed dan total_income memiliki nilai hilang yang sama.

In [ ]:
df_isna = df.fillna(0)

merubah semua nilai yang hilang menjadi 0 dan memasukan nya ke dalam variabel baru df_isna

In [ ]:
df_isna = df_isna[(df_isna['days_employed'] == 0) | 
                  (df_isna['total_income'] == 0)]
df_isna.count()

children            2174
days_employed       2174
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income        2174
purpose             2174
dtype: int64

pertama kita rubah semua nilai yang hilang pada data menjadi 0, setelah itu kita filter data menggunakan logika or pada kolom days_employed dan total_income yang memiliki nilai 0.
ternyata menghasilkan baris yang sama dengan total nilai yang hilang.

Kesimpulan:
nilai yang hilang pada kolom days_employes dan total_income berada di baris yang sama.


In [ ]:
persentase = df_isna['debt'].count() / df['debt'].count()
print(f'persentase nilai yang hilang adalah : {persentase:.0%}')

persentase nilai yang hilang adalah : 10%


nilai yang hilang sampai 10% dari seluruh data, nilai yang cukup besar.

coba kita cari lebih dalam mengapa banyak sekali nilai yang hilang dengan mengkategorikan nilai yang hilang dengan kolom yang sekiranya memiliki karakteristik tertentu nasabah.

In [ ]:
df_isna.groupby('purpose')['days_employed'].count().sort_values(ascending=False).head()

purpose
having a wedding                92
to have a wedding               81
wedding ceremony                76
construction of own property    75
housing transactions            74
Name: days_employed, dtype: int64

In [ ]:
df_isna.groupby('income_type')['days_employed'].count().sort_values(ascending=False).head()

income_type
employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: days_employed, dtype: int64

In [ ]:
df_isna.groupby('gender')['days_employed'].count().sort_values(ascending=False).head()

gender
F    1484
M     690
Name: days_employed, dtype: int64

1. Setelah melakukan perhitungan distribusi nilai yang hilang terhadap kolom debt, di dapat hampir 90% data hilang adalah nasabah yang tidak memiliki hutang pembayaran pinjaman, atau bisa dianggap adalah nasabah baru
2. jika nilai perhitungan distribusi nilai yang hilang terhadap kolom income_type, di dapat hampir 50% data hilang adalah employee, dimana seharusnya seorang employee memiliki total_income dan pengalaman dalam bekerja.
3. jika nilai perhitungan distribusi nilai yang hilang terhadap kolom purpose, didapat 3 teratas ditempati tentang pernikahan.

In [ ]:
df.describe(include='all')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
count,21525.000000,19351.000000,21525.000000,21525,21525.000000,21525,21525.000000,21525,21525,21525.000000,19351.000000,21525
unique,NaN,NaN,NaN,15,NaN,5,NaN,3,8,NaN,NaN,38
top,NaN,NaN,NaN,secondary education,NaN,married,NaN,F,employee,NaN,NaN,wedding ceremony
freq,NaN,NaN,NaN,13750,NaN,12380,NaN,14236,11119,NaN,NaN,797
mean,0.538908,63046.497661,43.293380,NaN,0.817236,NaN,0.972544,NaN,NaN,0.080883,26787.568355,NaN
std,1.381587,140827.311974,12.574584,NaN,0.548138,NaN,1.420324,NaN,NaN,0.272661,16475.450632,NaN
min,-1.000000,-18388.949901,0.000000,NaN,0.000000,NaN,0.000000,NaN,NaN,0.000000,3306.762000,NaN
25%,0.000000,-2747.423625,33.000000,NaN,1.000000,NaN,0.000000,NaN,NaN,0.000000,16488.504500,NaN
50%,0.000000,-1203.369529,42.000000,NaN,1.000000,NaN,0.000000,NaN,NaN,0.000000,23202.870000,NaN
75%,1.000000,-291.095954,53.000000,NaN,1.000000,NaN,1.000000,NaN,NaN,0.000000,32549.611000,NaN


In [ ]:
df_isna.describe(include = 'all')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
count,2174.000000,2174.0,2174.000000,2174,2174.000000,2174,2174.000000,2174,2174,2174.000000,2174.0,2174
unique,NaN,NaN,NaN,12,NaN,5,NaN,2,5,NaN,NaN,38
top,NaN,NaN,NaN,secondary education,NaN,married,NaN,F,employee,NaN,NaN,having a wedding
freq,NaN,NaN,NaN,1408,NaN,1237,NaN,1484,1105,NaN,NaN,92
mean,0.552438,0.0,43.632015,NaN,0.800828,NaN,0.975161,NaN,NaN,0.078197,0.0,NaN
std,1.469356,0.0,12.531481,NaN,0.530157,NaN,1.418220,NaN,NaN,0.268543,0.0,NaN
min,-1.000000,0.0,0.000000,NaN,0.000000,NaN,0.000000,NaN,NaN,0.000000,0.0,NaN
25%,0.000000,0.0,34.000000,NaN,0.250000,NaN,0.000000,NaN,NaN,0.000000,0.0,NaN
50%,0.000000,0.0,43.000000,NaN,1.000000,NaN,0.000000,NaN,NaN,0.000000,0.0,NaN
75%,1.000000,0.0,54.000000,NaN,1.000000,NaN,1.000000,NaN,NaN,0.000000,0.0,NaN


Jika dilakukan perbandingan antara data asli dengan data yang sudah di filter, kita menemukan persamaan bahwa : 
1. data terbanyak (Top) dari kolom income_type adalah employee
2. data terbanyak (Top) dari kolom purpose adalah terkait dengan wedding
3. data terbanyak (Top) dari kolom gender adalah F (female)

tetapi dengan adanya kesimpulan berikut, kita belum bisa membuat kesimpulan terhadap nilai yang hilang. dikarenakan masih kurang nya informasi yang lebih detail.

seperti mencari tahu data duplikat, kesalahan penulisan dan juga mengubah type data

###Transformasi Data

####Melihat nilai Unique dan Memperbaikinya

##### kolom education

In [ ]:
df['education'].unique()

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

dalam kolom education memiliki nilai unique yang ditulis dalam huruf besar dan juga huruf kecil. kita akan merubah penulisan menggunakan huruf kecil(lower capital) dan menampilkan hasilnya kembali

In [ ]:
df['education'] = df['education'].str.lower()
df['education'].unique()

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

#####kolom children

In [ ]:
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

\dalam kolom children memiliki nilai yang sedikit aneh, seperti -1, dan 20. bukan kah sedikit luar biasa jika memiliki 20 anak, apakah ada kesalahan input, kita akan mengecek nilai -1 dan 20 tersebut lebih detail.

In [ ]:
def df_children_unique(nilai):
  unique = df[df['children'] == nilai]['children'].count()
  return unique
print(f'jumlah nilai -1 pada kolom children : {df_children_unique(-1)}') 
print(f'jumlah nilai 20 pada kolom children : {df_children_unique(20)}')

jumlah nilai -1 pada kolom children : 47
jumlah nilai 20 pada kolom children : 76


jumlah nilai -1 pada kolom children sebanyak 47, dan jumlah nilai 20 pada kolom children sebanyak 76,
untuk nilai -1 akan kita rubah menjadi 1 dikarenakan terindikasi kesalahan input.
untuk nilai 20 setelah di selidiki lebih jauh, nilai 20 pada children tersebar pada nasabah di berbagai usia. yang meyakinkan bahwa nilai 20 pada kolom children terindikasi kesalah input yang seharusnya 2.

setelah itu kita tampilkan kembali nilai unique pada kolom children

In [ ]:
df.loc[df['children'] == 20, 'children'] = 2
df.loc[df['children'] == -1, 'children'] = 1
df['children'].unique()

array([1, 0, 3, 2, 4, 5])

#####Kolom Days Employee
pada kolom days employee berisi tentang berapa lama pengalaman bekerja nasabah dalam hari, yang menandakan bahwa nilai pada kolom days employee seharusnya adalah integer dan positif.

In [ ]:
df['days_employed'].unique()

array([-8437.67302776, -4024.80375385, -5623.42261023, ...,
       -2113.3468877 , -3112.4817052 , -1984.50758853])

di dapat hasil nilai unique pada kolom days employee adalah bilangan negatif dan bertype float.

kita akan merubah terlebih dahulu type nya menjadi integer. sebelum merubah type data nya kita rubah terlebih dahulu nilai yang hilang menjadi 0.

In [ ]:
df = df.fillna(0)

In [ ]:
df['days_employed'] = abs(df['days_employed'].astype('int64'))

In [ ]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,5623,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


dalam data days_employed beberapa memiliki nilai yang tidak wajar, seperti yang kita ketahui bahwa nilai max dalam kolom days_employed adalah 401755.400475	. nilai yang sanga besar, apabila di bagi dengan 365 hari dalam setahun 

In [ ]:
print('pengalaman kerja nasabah dalam tahun adalah {} tahun'.format(df['days_employed'].max()/365))

pengalaman kerja nasabah dalam tahun adalah 1100.6986301369864 tahun


wow dia telah bekerja selama 1100 tahun , tidak terbayangkan, ada kemungkinan salah memasukan input, 
selanjut nya, untuk nilai yang melebihi akan kita hitung jumlah nya dan berapa persen nilai yang anominil tersebut terhadap seluruh data. 

pertama kita akan menambahkan kolom baru berupa pengurangan dari usia nasabah dengan umur 17 tahun. (17 tahun adalah batas wajar seseorang dapat bekerja secara resmi) dan di kalikan 356 hari untuk membandingkan dengan kolom days_employee



In [ ]:
df['selisih_umur'] = (df['dob_years'] - 17) * 356

setelah menambahkan kolom baru, kita akan membandingkan antara selisih umur dengan days_employee, jika hasilnya adalah days_employed lebih besar di banding selisih umur, akan di masukan ke dalam sebuah variabel baru untuk mencari tahu berapa banyak nilai yang anomali.

In [ ]:
anomali_days_employed = df[df['days_employed'] > df['selisih_umur']]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,selisih_umur
1890,0,0,0,bachelor's degree,0,unmarried,4,F,employee,0,0.000,housing,-6052
2284,0,0,0,secondary education,1,widow / widower,2,F,retiree,0,0.000,property,-6052
4064,1,0,0,secondary education,1,civil partnership,1,M,business,0,0.000,housing renovation,-6052
6670,0,0,0,bachelor's degree,0,divorced,3,F,retiree,0,0.000,buy residential real estate,-6052
12403,3,0,0,secondary education,1,married,0,M,employee,0,0.000,transactions with commercial real estate,-6052
5014,0,0,0,secondary education,1,married,0,F,business,0,0.000,buy real estate,-6052
6411,0,0,0,bachelor's degree,0,civil partnership,1,F,retiree,0,0.000,wedding ceremony,-6052
13741,0,0,0,secondary education,1,civil partnership,1,F,employee,0,0.000,having a wedding,-6052
8574,0,0,0,secondary education,1,married,0,F,employee,0,0.000,property,-6052
19829,0,0,0,secondary education,1,married,0,F,employee,0,0.000,housing,-6052


In [ ]:
import pandas as pd
try:
  df = pd.read_csv('/content/credit_scoring_eng.csv')
except:
  df = pd.read_csv('/datasets/credit_scoring_eng.scv')

  #data credit scoring akan di simpan dalam variabel df

In [ ]:
df[df['dob_years'] == 0].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Secondary Education,1,married,0,F,retiree,0,11406.644,car
149,0,-2664.273168,0,secondary education,1,divorced,3,F,employee,0,11228.230,housing transactions
270,3,-1872.663186,0,secondary education,1,married,0,F,employee,0,16346.633,housing renovation
578,0,397856.565013,0,secondary education,1,married,0,F,retiree,0,15619.310,construction of own property
1040,0,-1158.029561,0,bachelor's degree,0,divorced,3,F,business,0,48639.062,to own a car
1149,0,-934.654854,0,secondary education,1,married,0,F,business,0,32296.389,buy real estate
1175,0,370879.508002,0,secondary education,1,married,0,F,retiree,0,50231.975,to get a supplementary education
1386,0,-5043.219890,0,bachelor's degree,0,married,0,M,civil servant,0,38483.779,purchase of a car
1890,0,NaN,0,bachelor's degree,0,unmarried,4,F,employee,0,NaN,housing
1898,0,370144.537021,0,secondary education,1,widow / widower,2,F,retiree,0,20384.043,to buy a car


In [43]:
import pandas as pd

df = pd.read_csv('/content/credit_scoring_eng.csv')

In [2]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


In [20]:
def purpose(row):
    wedding = df['purpose'].str.contains('wedding').astype('string')
    property = df['purpose'].str.contains('property').astype('string')
    for row in df['purpose']:
      if wedding == 'True':
        return 'wedding'
      elif property == 'True':
        return 'property ' 


In [21]:
df['type_purpose'] = df['purpose'].apply(purpose)

ValueError: ignored

In [48]:
df['wedding'] = df['purpose'].str.contains('wedding').astype('string')
df.loc[df['wedding'] == 'True','purpose_type'] = 'wedding'
df['property'] = df['purpose'].str.contains('property|house|housing|real estate' ).astype('string')
df.loc[df['property'] == 'True','purpose_type'] = 'property'
df['edu'] = df['purpose'].str.contains('education|educated|university').astype('string')
df.loc[df['edu'] == 'True','purpose_type'] = 'education'
df['car'] = df['purpose'].str.contains('car|cars').astype('string')
df.loc[df['car'] == 'True','purpose_type'] = 'car'
df = df.drop(columns = ['wedding','property','car','edu'])

In [46]:
df['purpose_type'].isna().sum()

0

In [49]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_type
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,property
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,car
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house,property
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,wedding
